# Run project pipeline
this notebook:
- checks channel audio activity
- extracts ADM metadata
- parses it
- writes spatial instructions CSV.
- more functionality to come

In [ ]:
# Cell: Environment / repo path check
import sys, os, shutil, subprocess
print("Python:", sys.executable, sys.version.splitlines()[0])
try:
    which_jupyter = shutil.which("jupyter") or "jupyter not on PATH"
    print("jupyter binary:", which_jupyter)
except Exception:
    pass

# Ensure repo root is on sys.path (assumes notebook is in notebooks/)
nb_dir = os.getcwd()
# if running from notebooks/, repo root is parent
repo_root = os.path.abspath(os.path.join(nb_dir, "..")) if os.path.basename(nb_dir) == "notebooks" else nb_dir
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)
print("Repo root:", repo_root)
print("sys.path[0]:", sys.path[0])

# quick git info (if present)
if os.path.exists(os.path.join(repo_root, ".git")):
    try:
        print("Git branch/last commit:")
        subprocess.run(["git", "-C", repo_root, "rev-parse", "--abbrev-ref", "HEAD"], check=False)
        subprocess.run(["git", "-C", repo_root, "log", "-n", "1", "--pretty=format:%h %s (%ci)"], check=False)
    except Exception as e:
        print("git info error:", e)
else:
    print(".git not found in repo root")

# Cell: Install missing python packages (if needed)
import importlib, sys, subprocess

reqs = ["lxml", "numpy", "soundfile"]  # core packages used by the repo
missing = []
for pkg in reqs:
    try:
        importlib.import_module(pkg)
    except Exception:
        missing.append(pkg)

if missing:
    print("Installing missing packages:", missing)
    subprocess.run([sys.executable, "-m", "pip", "install", "--upgrade"] + missing, check=True)
else:
    print("All core python packages present:", reqs)

# System dependency notes (macOS)
print("\nSystem dependency notes:")
print("- If you use PySoundFile / soundfile, you need libsndfile installed (macOS: `brew install libsndfile`).")
print("- If nb shows errors about JupyterLab build or missing schemas and you're trying to use JupyterLab, install Node.js (`brew install node`) and run `jupyter lab build` in your terminal (not in the notebook).")

from analyzeADM.extractMetadata import extractMetaData
from analyzeADM.parser import parseMetadata, getGlobalData
from packageADM.createRenderInfo import createSpatialInstructionsCSV
from analyzeADM.checkAudioChannels import exportAudioActivity

sourceADMFile = "sourceData/POE-ATMOS-FINAL.wav"


In [ ]:
print("\nChecking audio channels for content...")
exportAudioActivity(sourceADMFile)

print("Extracting ADM metadata from WAV file...")
extractedMetadata = extractMetaData(sourceADMFile, "processedData/currentMetaData.xml")

if extractedMetadata:
    xmlPath = extractedMetadata
    print(f"Using extracted XML metadata at {xmlPath}")
else:
    print("Using default XML metadata file")
    xmlPath = "data/POE-ATMOS-FINAL-metadata.xml"

print("Parsing ADM metadata...")
reformattedMetadata = parseMetadata(xmlPath, ToggleExportJSON=True, TogglePrintSummary=True)

print("\nCreating spatial instructions...")
spatialInstructions = createSpatialInstructionsCSV(
    processed_dir="processedData",
    output_path="forExport/spatialInstructions.csv"
)

print("\nDone")
